# Generate mesh in `gmsh`

In [1]:
import gmsh

In [2]:
l_1 = 50  # side length of small cube
l_2 = 250  # side length of big cube

In [3]:
# Initialize model
gmsh.initialize()
model = gmsh.model()

model.add("Omega")

In [4]:
small_cube = model.occ.add_box(0, 0, 0, l_1, l_1, l_1)
model.occ.translate([(3, small_cube)], -l_1/2, -l_1/2, -l_1/2)
big_cube = model.occ.add_box(0, 0, 0, l_2, l_2, l_2)
model.occ.translate([(3, big_cube)], -l_2/2, -l_2/2, -l_2/2)
out_dim, _ = model.occ.cut(
    [(3, big_cube)],
    [(3, small_cube)],
    removeObject=True,
    removeTool=False
)  # out_dim is a list of tuples (dim, tag). In this case it will be [(3, 3)]
air = out_dim[0][1]  # tag of the air
model.occ.synchronize()

In [5]:
boundaries_air = model.get_boundary([(3, air)], oriented=False)  # This extracts the boundary from the entity `air` (that has dimension 3)
boundaries_core = model.get_boundary([(3, small_cube)], oriented=False)  # This extracts the boundary from the entity `air` (that has dimension 3)

In [6]:
boundaries_tag_core = [dimTag[1] for dimTag in boundaries_core]

In [7]:
out_boundary_list = []
for dimTag in boundaries_air:
    if (tag:=dimTag[1]) not in boundaries_tag_core:
        out_boundary_list.append(tag)
out_boundary_list

[7, 8, 9, 10, 11, 12]

Define tags:

In [8]:
model.add_physical_group(3, [small_cube], name="core")
model.add_physical_group(3, [air], name="air")
model.add_physical_group(2, out_boundary_list, name="boundary")

3

Specify mesh options and generate the mesh.

In [9]:
model.mesh.set_size(model.get_entities(0), 10)  # assign mesh size 10 to all points
model.mesh.set_size(model.get_entities(2), 10)
eps=1e-3
points_in_core = model.get_entities_in_bounding_box(-25-eps, -25-eps, -25-eps, 25+eps, 25+eps, 25+eps)
model.mesh.set_size(points_in_core, 2)

In [10]:
model.mesh.generate(3)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 30%] Meshing curve 7 (Line)
Info    : [ 30%] Meshing curve 8 (Line)
Info    : [ 40%] Meshing curve 9 (Line)
Info    : [ 40%] Meshing curve 10 (Line)
Info    : [ 50%] Meshing curve 11 (Line)
Info    : [ 50%] Meshing curve 12 (Line)
Info    : [ 60%] Meshing curve 13 (Line)
Info    : [ 60%] Meshing curve 14 (Line)
Info    : [ 60%] Meshing curve 15 (Line)
Info    : [ 70%] Meshing curve 16 (Line)
Info    : [ 70%] Meshing curve 17 (Line)
Info    : [ 80%] Meshing curve 18 (Line)
Info    : [ 80%] Meshing curve 19 (Line)
Info    : [ 80%] Meshing curve 20 (Line)
Info    : [ 90%] Meshing curve 21 (Line)
Info    : [ 90%] Meshing curve 22 (Line)
Info    : [100%] Meshing curve 23 (Line)
Info    : [100%] Meshing curve 24 (Line)
I

Run GUI:

In [11]:
# gmsh.fltk.run()

Write mesh to file:

In [12]:
gmsh.write("./Omega.msh")
gmsh.finalize()

Info    : Writing './Omega.msh'...
Info    : Done writing './Omega.msh'
